Après plusieurs entretiens qui ont pour la majorité abouti à une mise en suspension du processsus de recrutement avec comme argument principale : 
"Nous sommes très intéressés par votre profil mais malheureusement nous avons peu de missions pour le moment, nous vous recontactons dès que nous avons quelcque chose et de votre côté tenez-nous au courant si vous avez des propositions."

Ayant atteint mon objectif personnel de renforcer mes compétences en python à un niveau proche de celui que j'ai en R (7 ans d'expérience) grâce notamment aux deux livres suivants:

1. Apprendre à programmer en python
2. Machine learning avec python

En attendant deux autres retours d'entretiens, j'ai acquis un troisième livre (Deep Learning avec TensorFlow) et je m'apprêtais poursuivre mon apprentissage du deep learning initié il y a plus d'un an à travers la spécialisation deep learning de *deepleraning.ai* sur coursera lorsque je me suis dit que je voulais aller plus loin que la simple lecture du livre et la résolution des exercices proposés.

Après quelques minutes de réflexion j'ai assez rapidement trouvé un sujet qui m'occuperait vraiment l'esprit et qui me ferait grandement progresser dans mon métier de data scientist. J'ai décidé de construire un modèle de machine translation pour traduire mon rapport de stage de fin d'étude.

In [1]:
import collections

import helper
import numpy as np
import project_tests as tests

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [2]:
english_sentences = helper.load_data('small_vocab_en')
french_sentences = helper.load_data('small_vocab_fr')

In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [4]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '""'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()

print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '""'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')
print()

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is"","".""in""it""during""the""but""and""sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est""."",""en""il""les""mais""et""la""parfois"



In [5]:
def tokenize(x):
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

text_sentences = ['The quick brown fox jumps over the lazy dog .',
                  'By Jove , my quick study of lexicography won a prize .',
                  'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input: {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input: The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input: By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input: This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [6]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input: {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input: [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input: [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input: [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [7]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = \
    preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [8]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length, 
    english_vocab_size+1, 
    french_vocab_size+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 77s 698us/sample - loss: 3.5096 - acc: 0.4149 - val_loss: 2.5590 - val_acc: 0.4593
Epoch 2/10
110288/110288 [==============================] - 69s 630us/sample - loss: 2.4364 - acc: 0.4697 - val_loss: 2.3108 - val_acc: 0.4838
Epoch 3/10
110288/110288 [==============================] - 67s 609us/sample - loss: 2.1916 - acc: 0.5092 - val_loss: 2.0631 - val_acc: 0.5237
Epoch 4/10
110288/110288 [==============================] - 82s 742us/sample - loss: 1.9536 - acc: 0.5468 - val_loss: 1.8576 - val_acc: 0.5687
Epoch 5/10
110288/110288 [==============================] - 96s 871us/sample - loss: 1.7942 - acc: 0.5742 - val_loss: 1.7376 - val_acc: 0.5787
Epoch 6/10
110288/110288 [==============================] -

In [ ]:
from tensorflow.keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1])
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    
    return model
#tests.test_embed_model(embed_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embeded_model = embed_model(tmp_x.shape, max_french_sequence_length, english_vocab_size+1, french_vocab_size+1)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

In [11]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.1), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

#tests.test_bd_model(bd_model)

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bidi_model = bd_model(tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index)+1, 
                      len(french_tokenizer.word_index)+1)

bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/20
110288/110288 [==============================] - 220s 2ms/sample - loss: 2.7350 - acc: 0.4954 - val_loss: 1.8011 - val_acc: 0.5739
Epoch 2/20
110288/110288 [==============================] - 226s 2ms/sample - loss: 1.6648 - acc: 0.5848 - val_loss: 1.4867 - val_acc: 0.6076
Epoch 3/20
110288/110288 [==============================] - 242s 2ms/sample - loss: 1.4590 - acc: 0.6114 - val_loss: 1.3664 - val_acc: 0.6190
Epoch 4/20
110288/110288 [==============================] - 211s 2ms/sample - loss: 1.3509 - acc: 0.6279 - val_loss: 1.3006 - val_acc: 0.6338
Epoch 5/20
110288/110288 [==============================] - 208s 2ms/sample - loss: 1.2779 - acc: 0.6404 - val_loss: 1.2557 - val_acc: 0.6369
Epoch 6/20
110288/110288 [==============================] - 184s 2ms/sample - loss: 1.2249 - acc: 0.6512 - val_loss: 1.2346 - val_acc: 0.6367
Epoch 7/20
110288/110288 [==============================] - 182s 2ms/sample - loss: 1.1809 - acc:

In [12]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate =1e-3
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape[1:], return_sequences=False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

#tests.test_encdec_model(encdec_model)

tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index)+1,
                              len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/20
110288/110288 [==============================] - 168s 2ms/sample - loss: 2.9773 - acc: 0.4427 - val_loss: 2.4525 - val_acc: 0.4849
Epoch 2/20
110288/110288 [==============================] - 188s 2ms/sample - loss: 2.2846 - acc: 0.4989 - val_loss: 2.1141 - val_acc: 0.5046
Epoch 3/20
110288/110288 [==============================] - 199s 2ms/sample - loss: 2.0115 - acc: 0.5119 - val_loss: 1.9279 - val_acc: 0.5199
Epoch 4/20
110288/110288 [==============================] - 197s 2ms/sample - loss: 1.8617 - acc: 0.5373 - val_loss: 1.7930 - val_acc: 0.5530
Epoch 5/20
110288/110288 [==============================] - 180s 2ms/sample - loss: 1.7394 - acc: 0.5623 - val_loss: 1.6855 - val_acc: 0.5697
Epoch 6/20
110288/110288 [==============================] - 187s 2ms/sample - loss: 1.6423 - acc: 0.5730 - val_loss: 1.5950 - val_acc: 0.5802
Epoch 7/20
110288/110288 [==============================] - 196s 2ms/sample - loss: 1.5631 - acc:

In [13]:
from tensorflow.keras.models import Sequential
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    model = Sequential()
    
    model.add(Embedding(input_dim=english_vocab_size, output_dim=128, input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256, return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    learning_rate = 0.005
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    
    return model

#tests.test_model_final(model_final)

print('Final Model Loaded')

Final Model Loaded


In [ ]:
def final_prediction(x, y, x_tk, y_tk):
    
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index)+1, 
                       len(french_tokenizer.word_index)+1)
    model.fit(tmp_X, preproc_french_sentences, batch_size=1024, epochs=17, validation_split=0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    return model

model = final_prediction(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
y_id_to_word[0] = '<PAD>'
# TO DO  implent return here
sentence = 'he saw a old yellow truck'
sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
sentences = np.array([sentence[0], preproc_english_sentences[0]])
predicitions = model.predict(sentences, len(sentences))
    
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in preproc_french_sentences[0]]))


Train on 110288 samples, validate on 27572 samples
Epoch 1/17
 72704/110288 [==================>...........] - ETA: 3:59 - loss: 2.3765 - acc: 0.4842

# References

Susan li : Neural Machine Translation with Python (https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd)

In [ ]:
tmp_x